In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from selenium.webdriver.common.keys import Keys
import json
from opencc import OpenCC
import pandas as pd
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
import re
from tqdm import tqdm

In [2]:
tag = pd.read_excel('./merchant_top10000.xlsx')
brands_10000 = []

for i in range(len(tag)):
  brands_10000.append(tag['merchant_cleasing_tag'][i])

brands_10000

['momo購物網',
 '統一超商',
 '優食台灣股份有限公司',
 'foodpanda',
 '樂購蝦皮',
 '中油',
 '全家便利商店',
 '富邦人壽',
 '全聯app',
 '台灣大哥大',
 '全聯福利中心',
 '電話費',
 '家樂福',
 '麥當勞',
 '悠遊卡自動加值',
 '優步',
 '高鐵',
 '富邦產物',
 '新光三越',
 '萊爾富',
 '寶雅',
 '遠東百貨',
 '50嵐',
 '屈臣氏',
 '星巴克',
 'linetaxi',
 '遠傳電信',
 '橘子支付',
 '台灣大車隊',
 '大潤發',
 '全盈支付金融科技股份有限公司',
 'line禮物',
 '美廉社',
 '康是美',
 '全國加油站',
 '可不可熟成紅茶',
 '台灣之星',
 'ikea宜家家居',
 '城市車旅',
 '八方雲集',
 'louisa coffee',
 '清心福全',
 'pchome',
 '台茂購物中心',
 '大樹連鎖藥局',
 '臺灣鐵路管理局網路購票',
 '迷客夏',
 '遠東sogo百貨',
 '誠品',
 '愛買',
 '東森得易購',
 '和雲行動服務',
 '蝦皮支付',
 'qburger',
 '麻古茶坊',
 '一卡通自動加值',
 '嘟嘟房',
 'globalmall',
 '爭鮮迴轉壽司',
 'uniqlo',
 '電費',
 '東森購物',
 'mitsui outlet',
 '遠東巨城購物中心',
 '摩斯漢堡',
 '必勝客',
 '雅虎奇摩購物中心',
 'worldgym',
 '亞太電信',
 '統一時代百貨',
 'yoxi',
 '麥味登',
 '肯德基',
 'coco',
 '小北百貨',
 '新東陽',
 '漢神巨蛋購物廣場',
 'decathlon迪卡儂',
 'global mall',
 '台灣自來水',
 'ok line pay',
 '統一精工',
 '北市停車費',
 '停車大聲公',
 '網購-91app',
 '京站時尚廣場',
 '美麗華百樂園',
 '電信資費',
 '85度c',
 'kfc',
 't startel',
 'goshare',
 'gogoro network',
 '交通部臺灣鐵路管理局',
 '義美股份有限

In [3]:
#取要爬的區間
#改的特店名：夢時代?統一時代 =>夢時代統一時代
brands1 = brands_10000[:50]
brands2 = brands_10000[50:100]
brands3 = brands_10000[100:112]
brands4  = brands_10000[112:200]
brands5  = brands_10000[200:289]
brands6 = brands_10000[289:300]
brands = brands_10000[300:400]

In [5]:

cc = OpenCC('s2tw')

#找到本瀏覽器的user agent
url = 'https://httpbin.org/headers'
browser = webdriver.Chrome('./chromedriver_win32/chromedriver')
browser.get(url)
user_agent = browser.page_source
browser.close()
try:
    match = re.search(r"\"User-Agent\":\s+\"(.*?)\",", user_agent).group(1)
except:
    match = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
print(match)


#記得創一個content的資料夾
fp_output = r'./content/result_{}.json'

#所有要爬的特店像這樣放在這個list中
#-------------------------------------------#
# brands = ['八方雲集', '我大巴', '舒服雷']
#-------------------------------------------#
#跟在特店名後面一起搜尋
keyword = '品牌'

#每個特店要爬幾頁
pagesss = 4

forloop_count = 0
for emo in tqdm(range(len(brands))):
    data_dic = {}
    page_num_dic = {}
    print(f'第{emo+1}間特店：{brands[emo]}，總共有{len(brands)}家')
    with open(f'./process.txt', "a+") as f:
        f.write(f'第{emo+1}間特店：{brands[emo]}，總共有{len(brands)}家\n')
    forloop_count+=1
    if forloop_count%10==0: #每10家休息200秒
        time.sleep(20) 
    time.sleep(10) #每家休息幾秒
    try:
        sleep(1)
        #options.add_argument("headless")
        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument(
            f'user-agent={match}')
        options.add_argument('--disable-blink-features=AutomationControlled')
        driver = webdriver.Chrome('./chromedriver_win32/chromedriver',options=options)
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
                })
            """
            })
        
        
        driver.implicitly_wait(5)
        driver.get('https://www.google.com') #navigating to google
        search_gog = driver.find_element(By.CLASS_NAME, "gLFyf")
        search_gog.send_keys(f'{brands[emo]} {keyword}')
        search_gog.send_keys(Keys.RETURN)
        li = []
        for i in range(pagesss):  # page number
            for des in driver.find_elements(By.CLASS_NAME, "VwiC3b.yXK7lf.MUxGbd.yDYNvb.lyLwlc.lEBKkf"):
                text = des.text
                li.append(cc.convert(text))
            try:
                next_links = driver.find_elements(By.LINK_TEXT, '下一頁')# note it is find_elements with an s
                next_links[0].location_once_scrolled_into_view
                actions = ActionChains(driver)
                time.sleep(random.uniform(3,6))
                # 滑鼠先移到 submitBtn 上，然後再點擊 submitBtn
                actions.move_to_element(next_links[0]).click(next_links[0])
                actions.perform()
            except:
                break

        data_dic[f'{brands[emo]}'] = li
        print(data_dic[f'{brands[emo]}'])
        page_num_dic[f'{brands[emo]}'] = i
        
        with open(fp_output.format(brands[emo]), 'w') as f:
             f.write(json.dumps(data_dic))
    finally:
#         input('pausing (hit enter to terminate) ...')
        driver.quit()

<ipython-input-5-7973a1320fdc>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('./chromedriver_win32/chromedriver')


Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36


  0%|          | 0/100 [00:00<?, ?it/s]

第1間特店：誠品信義旗艦店，總共有100家


<ipython-input-5-7973a1320fdc>:47: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver_win32/chromedriver',options=options)


...

 98%|█████████▊| 98/100 [1:25:26<01:38, 49.02s/it]

第99間特店：漢神名店百貨股份，總共有100家
['漢神百貨提供齊全的商品，進駐國際精品、美妝保養、休閒運動、孕婦與嬰幼童、主題餐廳、3C數位電器...等百大品牌，持續為廣大消費者，創造一個精緻、豐富、且食、 衣、 ...', '宜得利、OSIM、DYSON、伊聖詩⋯等共62 個居家生活品牌，盡在漢神百貨，申辦漢神百貨聯名卡消費再享購物、停車與漢來大飯店專享優惠。', 'BVLGARI. BVLGARI寶格麗成立於1884年，從一家珠寶店，現已逐漸發展為象徵義大利卓越品質的華麗珠寶品牌。 · Cartier. Cartier創立於1847年的巴黎，傳承著世家（Maison）獨有 ...', '精緻典雅的漢神百貨，是高雄亞洲新灣區的核心地標，館內世界頂級精品品牌雲集，是南臺灣國際經典時尚的指標，貼心的服務，讓每位貴賓盡享尊榮。', 'GUCC、BALENCIAGA、BAO BAO ISSEY MIYAKE、COACH⋯等共14 個品牌，盡在漢神百貨3F 世界精品。本樓層另設有女性專用補妝室，歡迎多加使用。', 'Blumarine、MONTBLANC、Paul Smith、POLO RALPH LAUREN⋯等共24 個品牌，盡在漢神百貨4F 國際名品。本樓層另設有卡友服務中心、貴賓廳、中國信託商業銀行ATM， ...', 'LOUIS VUITTON、BURBERRY、BVLGARI、GUCCI⋯等共41 個國際精品品牌，盡在漢神巨蛋，申辦漢神巨蛋聯名卡消費再享購物、餐飲、停車等專享優惠。', '微風集團於2015年創立複合式童裝品牌，網羅全球精品童裝品牌，BALMAIN、Chloé、Dolce & Gabbana、FENDI、Givenchy、Hugo boss、LANVIN、Marc ...', 'BURBERRY、DIOR、LOUIS VUITTON、PRADA⋯等共7 個品牌，盡在漢神百貨2F 世界精品。本樓層另設有女性專用補妝室，歡迎多加使用。', '誠品生活expo、10/10 HOPE、Sealy席伊麗床墊、德國雙人⋯等共48 個居家生活品牌，盡在漢神巨蛋，申辦漢神巨蛋聯名卡消費再享購物、餐飲、停車等專享優惠。', '漢神巨蛋地上8 層至地下1 層共9 個樓層，超過500 家品牌名店，多間南臺灣獨家餐廳，鼎泰豐、ZARA、H&M、海底撈、Uniqlo、

 99%|█████████▉| 99/100 [1:26:13<00:48, 48.39s/it]

第100間特店：金興發股份有限公，總共有100家
['金興發經營團隊秉持著JSF(JSF = JUSTICE SERVICE FUTURE)的理念與原則，除了持續推動企業全方位成長，堅持站在消費者的立場，針對商品的價格與品質，為消費者把關，更致力 ...', '金興發擁有多種精選辦公文具、學生文具及生活雜貨...等豐富商品打造您的質感生活。 未來將打造全新的金興發，以提供複合式文化商品，讓生活充滿儀式感！', 'オレンジページ日本雜誌│特刊20237/2-7/17. LOVE臺灣Book 雜貨天國專輯介紹來臺必買臺灣特色文具、生活雜貨，營業到深夜的金興發(南西店)，是來臺購物之地 ; 100年7月號第 ...', '金興發經營團隊秉持著JSF的理念與原則，除了持續推動企業全方位成長，堅持站在消費者的立場，針對商品的價格與品質，為消費者把關，更致力讓每位來店消費的顧客都能感受「 ...', '金興發，發現生活線上購物提供新品上市相關優惠與推薦商品價格可供挑選，線上選購熱門活動相關商品輕鬆簡單，更多新品上市就在金興發，發現生活線上購物.', '金興發，發現生活線上購物提供【花仙子】相關優惠與推薦商品價格可供挑選，線上選購【指定品牌】滿額贈相關商品輕鬆簡單，更多【花仙子】就在金興發，發現生活線上 ...', '2020年1月3日 — 臺灣30年在地老品牌金興發生活百貨與美食外送平臺「foodpanda」合作，目前開放七間北區門市南西、永和、士林、城中、三和、通化、臺大、師大三讓消費 ...', '公司簡介. 【品牌介紹】 金興發拼出生活用品王國民國79年，在走投無路的窘境下，讓當時經商失敗的詹老闆夫婦，開始四處奔波擺路邊攤的生活。「閒時不閒日」是在路邊攤 ...', '永麗生活百貨有限公司已在香港成立了七年, 現分別設立香港、臺灣兩地公司。為保養品和食品出入口商和批發商。香港公司有幸成為三個韓國保養品品牌的港澳總代理。由香港公司 ...', '金興發股份有限公司(金興發生活百貨)-臺北市中山區-超市/百貨/連鎖量販店-(02)77012588-服務業.', '秉持著理念與原則，除了持續推動企業全方位成長； 堅持站在消費者的立場，針對商品的價格與品質，為消費者把關。 致力讓每位來店消費的顧客都能感受「輕鬆購物省到家」的 ...', '... 讓當時經商失敗的詹老闆夫婦，開始

100%|██████████| 100/100 [1:27:21<00:00, 52.41s/it]
